<a href="https://colab.research.google.com/github/abhipise2704/RAG_LlamaIndex_Azure_OpenAI/blob/main/Llama_Index_Blog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Health Document Summarizer using RAG,LlamaIndex and Azure OpenAI

## 1. Installing Libraries

In [5]:
!pip install openai llama-index==0.9.48
!pip install pypdf==4.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 9.9 MB/s eta 0:00:00


## 2. Enviroment Varibles

In [7]:
import os
api_key = "azure openai api_key"
api_version = "version of azure openai llm model e.g.: 2023-07-01-preview"
azure_endpoint = 'https://{name_of_endpoint}.openai.azure.com'

## 3. Model Configuration of AzureOpenai and Embedding

In [9]:
from llama_index.llms import AzureOpenAI
from llama_index.embeddings import AzureOpenAIEmbedding
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

llm = AzureOpenAI(
    deployment_name="deployment_name",
    model="model name",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
    temprature=0.6
)
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

from llama_index import set_global_service_context

service_context = ServiceContext.from_defaults(chunk_size=200,
                                               llm=llm,
                                               embed_model=embed_model,
)

set_global_service_context(service_context)

## 4. PDF Parsing

In [10]:
documents = SimpleDirectoryReader(
    input_files=["/content/Report5.15 yo male with Ewings Sarcoma, pediatric tumor with good PS.pdf"]
).load_data()
index = VectorStoreIndex.from_documents(documents)

## 5. Output: Summary of Health Document  

In [18]:
import pprint
query = """Create a concise summary of a cancer patient's treatment plan. The document contains complex medical procedures and terminology, which need to be translated into simple, layman's terms. Focus on the following points to ensure the summary is accessible and informative:

Diagnosis: Briefly explain the type of cancer diagnosed in simple terms.
Treatment Objective: State the goal of the treatment plan (e.g., to remove cancer, reduce symptoms, improve quality of life) in understandable language.
Main Treatments: List the primary treatments proposed (such as surgery, chemotherapy, radiation) and describe their purpose in a way that anyone can understand.
Treatment Process: Summarize how the treatment will be administered and the expected timeline, avoiding medical jargon.
Side Effects: Mention common side effects of the treatments in plain language, and how they might affect the patient's daily life.
Follow-up Care: Outline the follow-up care plan, including any additional tests or treatments that may be needed after the main treatment is completed.
Support Services: Highlight any support services offered (like counseling or support groups) that help with emotional, physical, or financial aspects of cancer treatment.
Ensure the summary is empathetic and offers reassurance, emphasizing the support available to the patient throughout their treatment journey. The goal is to make the treatment plan understandable and less intimidating to the patient and their loved ones.
"""

query_engine = index.as_query_engine(similarity_top_k=5)
answer = query_engine.query(query)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(f'Summry: {answer}')

("Summry: A 15-year-old male has been diagnosed with Ewing's Sarcoma, a type "
 'of pediatric tumor. The goal of the treatment plan is to manage symptoms and '
 "improve the patient's quality of life. The primary treatments proposed are "
 'pain management, which can be achieved through medication such as '
 'acetaminophen and opioids, and symptom management through the expertise of '
 'palliative care clinicians. The treatment process will be administered based '
 'on the type, source, severity, and duration of pain. Common side effects of '
 'the treatments include mild to severe pain, which can be managed through '
 "medication. Follow-up care will be provided based on the patient's clinical "
 'urgency. Support services such as counseling and support groups are '
 'available to help with emotional, physical, or financial aspects of cancer '
 'treatment. The patient and their loved ones can be reassured that they will '
 'receive the necessary support throughout their treatment jour